In [1]:
!pip install numpy
!pip install pandas
!pip install transformers
!pip install torch
!pip install datasets
!pip install sklearn

Looking in indexes: https://pypi.python.org/simple/
Looking in indexes: https://pypi.python.org/simple/
Looking in indexes: https://pypi.python.org/simple/
Looking in indexes: https://pypi.python.org/simple/
Looking in indexes: https://pypi.python.org/simple/
Looking in indexes: https://pypi.python.org/simple/


In [13]:
import numpy as np
import pandas as pd
from transformers.data import datasets
from datasets import load_dataset
from datasets import Dataset, DatasetDict
from torch.utils.data import DataLoader
from transformers import EarlyStoppingCallback, AutoTokenizer, DataCollatorWithPadding, AdamW
from transformers import TrainingArguments, Trainer
import torch
from sklearn.model_selection import train_test_split

from utils import load_data

# Preprocessing

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
def load_whole_ds(DATA_PATH = "../data/"):

    # Load data from Set A, B and EX
    train_A, test_A, val_A = load_data(DATA_PATH, version="A", suffix="_kw", reduce_memory=True)
    train_B, test_B, val_B = load_data(DATA_PATH, version="B", suffix="_kw", reduce_memory=True)
    train_EX, test_EX, val_EX = load_data(DATA_PATH, version="EX", suffix="_kw", reduce_memory=True)

    #We concatenate the 3 different sets (A, B, EX):
    train_df = pd.concat([train_A, train_B, train_EX])
    test_df = pd.concat([test_A, test_B, test_EX])
    valid_df = pd.concat([val_A, val_B, val_EX])

    # All the data
    all_df = pd.concat([train_df, test_df, valid_df])
    
    return train_df, test_df, valid_df

In [5]:
def load_small_ds(DATA_PATH = "../data/"):

    small_df = pd.read_csv(DATA_PATH + 'small_dataset.csv')
    all_x = small_df.iloc[:,-1]
    all_y = small_df.iloc[:,:-1]
    
    X_train, X_test, y_train, y_test = train_test_split(all_x, all_y,
    test_size=0.2, shuffle = True, random_state = 8)

    # Use the same function above for the validation set
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
    test_size=0.25, random_state= 8) # 0.25 x 0.8 = 0.2
    
    train_df = y_train
    train_df['input_raw'] = X_train
    test_df = y_test
    test_df['input_raw'] = X_test
    val_df = y_val
    val_df['input_raw'] = X_val
    
    return train_df, test_df, val_df

In [6]:
%%time
DATA_PATH = "/home/jovyan/workbench-shared-folder/canary-project/Paula_internship/data/"
# Choose whether you want to load the whole dataset for the experiment
train_df, test_df, valid_df = load_whole_ds(DATA_PATH)
# train_df, test_df, valid_df = load_small_ds(DATA_PATH)

trds = Dataset.from_pandas(train_df)
vds = Dataset.from_pandas(valid_df)
teds = Dataset.from_pandas(test_df)

full_ds = DatasetDict()

full_ds['train'] = trds
full_ds['validation'] = vds
full_ds['test'] = teds

/home/jovyan/workbench-shared-folder/canary-project/Paula_internship/utils.py:46: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  train = pd.read_csv(f"{path}set_{version}_train{suffix}.csv", engine='python', error_bad_lines=False)
Skipping line 157634: unexpected end of data
/home/jovyan/workbench-shared-folder/canary-project/Paula_internship/utils.py:47: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  test = pd.read_csv(f"{path}set_{version}_test{suffix}.csv", engine='python', error_bad_lines=False)
/home/jovyan/workbench-shared-folder/canary-project/Paula_internship/utils.py:48: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  val = pd.read_csv(f"{path}set_{version}_val{suffix}.csv", engine='python', error_bad

CPU times: user 47.6 s, sys: 6.88 s, total: 54.5 s
Wall time: 59.9 s


In [43]:
train_df.head()

,pui,input_raw,input,human,controlled study,major clinical study,clinical article,mouse,case report,rat,...,human tissue,normal human,human experiment,observational study,pilot study,longitudinal study,retrospective study,case control study,cohort analysis,cross-sectional study
0,2007679937,"Frailty Index, Hospital Admission and Number o...",frailty index hospital admission number days s...,1,1,1,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,2010763871,Taohong siwu decoction attenuates myocardial f...,taohong siwu decoction attenuates myocardial f...,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,634109200,'It's all about patient safety': An ethnograph...,patient safety ethnographic study pharmacy sta...,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2008607441,Influence of ruthenium doping on UV- and visib...,influence ruthenium doping uv visiblelight pho...,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2005774817,Cabs1 maintains structural integrity of mouse ...,cabs maintains structural integrity mouse sper...,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
try:
    labels = [label for label in full_ds['train'].features.keys() if label not in ['pui', 'input_raw', 'input', '__index_level_0__', 'Unnamed: 0']]
except:
    labels = [label for label in full_ds['test'].features.keys() if label not in ['pui', 'input_raw', 'input', '__index_level_0__', 'Unnamed: 0']]

id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

len(labels)

52

In [8]:
%%time
#pretrained_model_name = "microsoft/xtremedistil-l6-h256-uncased"  # for xtremedistil transformer
pretrained_model_name = "allenai/scibert_scivocab_cased" # for scibert
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)

def preprocess_data(examples):
  # take a batch of texts
  text = examples["input_raw"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=512)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()
  
  return encoding

CPU times: user 412 ms, sys: 18.8 ms, total: 430 ms
Wall time: 3.34 s


In [9]:
%%time
try:
    encoded_dataset = full_ds.map(preprocess_data, batched=True, remove_columns=full_ds['train'].column_names)
except:
    encoded_dataset = full_ds.map(preprocess_data, batched=True, remove_columns=full_ds['test'].column_names)

  0%|          | 0/304 [00:00<?, ?ba/s]

  0%|          | 0/51 [00:00<?, ?ba/s]

  0%|          | 0/152 [00:00<?, ?ba/s]

CPU times: user 15min 57s, sys: 15.5 s, total: 16min 12s
Wall time: 6min 3s


In [10]:
print(encoded_dataset.keys())
print(encoded_dataset['test'][0].keys())

dict_keys(['train', 'validation', 'test'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [33]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(pretrained_model_name, 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)


loading configuration file https://huggingface.co/allenai/scibert_scivocab_cased/resolve/main/config.json from cache at /home/jovyan/.cache/huggingface/transformers/62ac366c3e40ed8952fcec53445ee752300aef550f61adff8f5b3485a268492b.bcde2c1ebafc440ded7d6525db15f6f30a50849b17cfb77242a97a8036b82861
Model config BertConfig {
  "_name_or_path": "allenai/scibert_scivocab_cased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "human",
    "1": "controlled study",
    "2": "major clinical study",
    "3": "clinical article",
    "4": "mouse",
    "5": "case report",
    "6": "rat",
    "7": "randomized controlled trial",
    "8": "clinical trial",
    "9": "multicenter study",
    "10": "diagnostic test accuracy study",
    "11": "systematic review",
    "12": "double blind procedure",
    "13": "meta analysis",
    "14": "controlled clinical trial",
    "15": "crossover pro

# Loss functions definition

In [14]:
# Use custom loss

class F1Trainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
    
        S=-1
        E=0
        y = labels
        # Sigmoid hyperparams:
        b = torch.tensor(S)
        c = torch.tensor(E)

        # Calculate the sigmoid
        sig = 1 / (1 + torch.exp(b * (logits + c)))
        tp = torch.sum(sig * y, dim=0)
        fp = torch.sum(sig * (1 - y), dim=0)
        fn = torch.sum((1 - sig) * y, dim=0)

        sigmoid_f1 = 2*tp / (2*tp + fn + fp + 1e-16)
        cost = 1 - sigmoid_f1
        macroCost = torch.mean(cost)

        return (macroCost, outputs) if return_outputs else macroCost

In [15]:
gamma = 2
alpha = 0.75

class FLTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        
        y = labels
        y_hat = outputs.get("logits")
        
        BCEWithLogitsLoss = torch.nn.BCEWithLogitsLoss(reduction='none')
        bce = BCEWithLogitsLoss(y_hat, y)
        
        pt = torch.exp(-bce)
        
        alpha_factor = y * alpha + (1 - y) * (1 - alpha)
        modulating_factor = torch.pow((1.0 - pt), gamma)

        focal_loss = torch.mean(alpha_factor * modulating_factor * bce)
#         focal_loss = torch.mean(modulating_factor * ce)

        return (focal_loss, outputs) if return_outputs else focal_loss

In [16]:
gamma_neg=4
gamma_pos=1
clip=0.05
eps=1e-8
disable_torch_grad_focal_loss = True

class ASLTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        y = labels
        y_hat = outputs.get("logits")
        
        # Calculating Probabilities
        x_sigmoid = torch.sigmoid(y_hat)
        xs_pos = x_sigmoid
        xs_neg = 1 - x_sigmoid

        # Asymmetric Clipping
        if clip is not None and clip > 0:
            xs_neg = (xs_neg + clip).clamp(max=1)

        # Basic CE calculation
        los_pos = y * torch.log(xs_pos.clamp(min=eps))
        los_neg = (1 - y) * torch.log(xs_neg.clamp(min=eps))
        loss = los_pos + los_neg

        # Asymmetric Focusing
        if gamma_neg > 0 or gamma_pos > 0:
            if disable_torch_grad_focal_loss:
                torch.set_grad_enabled(False)
            pt0 = xs_pos * y
            pt1 = xs_neg * (1 - y)  # pt = p if t > 0 else 1-p
            pt = pt0 + pt1
            one_sided_gamma = gamma_pos * y + gamma_neg * (1 - y)
            one_sided_w = torch.pow(1 - pt, one_sided_gamma)
            if disable_torch_grad_focal_loss:
                torch.set_grad_enabled(True)
            loss *= one_sided_w
        
        asl_loss = -loss.mean()
        
        return (asl_loss, outputs) if return_outputs else asl_loss

In [17]:
# Use custom loss for HAMMING LOSS

class HLTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
    
        S=-1
        E=0
        y = labels
        # Sigmoid hyperparams:
        b = torch.tensor(S)
        c = torch.tensor(E)

        # Calculate the sigmoid
        sig = 1 / (1 + torch.exp(b * (logits + c)))

        fp = torch.sum(sig * (1 - y), dim=0)
        fn = torch.sum((1 - sig) * y, dim=0)
        
        hamm_loss = (fp + fn) / torch.sum(labels, dim=0).clamp(min=0.5)  #avoid dividing by 0 if there is no label for the class
        
        macroCost = torch.mean(hamm_loss)

        return (macroCost, outputs) if return_outputs else macroCost

In [20]:
# Use custom loss
def get_classWeights():
    train_labels = train_df.iloc[:, 3:]
    tot = sum(train_labels.sum(axis=0))
    weight = 1 - (train_labels.sum(axis=0) / tot)
    
    return torch.tensor(weight)

weights = get_classWeights().to(device)
weights = weights**2

class F1weightTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
    
        S=-1
        E=0
        y = labels
        # Sigmoid hyperparams:
        b = torch.tensor(S)
        c = torch.tensor(E)

        # Calculate the sigmoid
        sig = 1 / (1 + torch.exp(b * (logits + c)))
        tp = torch.sum(sig * y, dim=0)
        fp = torch.sum(sig * (1 - y), dim=0)
        fn = torch.sum((1 - sig) * y, dim=0)

        sigmoid_f1 = 2*tp / (2*tp + fn + fp + 1e-16)
        cost = 1 - sigmoid_f1
        weighted_cost = torch.mul(cost, weights)
        macroCost = torch.mean(weighted_cost)

        return (macroCost, outputs) if return_outputs else macroCost

In [21]:
class F1learnTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
    
        S=-2
        E=0
        y = labels
        # Sigmoid hyperparams:
        b = torch.tensor(S)
        c = self.model.F1thr # Learnable parameter!!!!!!!!!!!!!!!! :)
#         c = torch.tensor(E)

        # Calculate the sigmoid
        sig = 1 / (1 + torch.exp(b * (logits + c)))
        tp = torch.sum(sig * y, dim=0)
        fp = torch.sum(sig * (1 - y), dim=0)
        fn = torch.sum((1 - sig) * y, dim=0)

        sigmoid_f1 = 2*tp / (2*tp + fn + fp + 1e-16)
        cost = 1 - sigmoid_f1
        macroCost = torch.mean(cost)

        return (macroCost, outputs) if return_outputs else macroCost

# Trainer

In [22]:
args = TrainingArguments(
    f"scibert-cased",
    evaluation_strategy = 'steps',
    eval_steps = 500,
    save_strategy = "steps",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='f1_micro',
    report_to="none",
    fp16=True,
    gradient_checkpointing=True,
    #gradient_accumulation_steps=256
)

In [23]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
# source: https://jesuslea l.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    f1_macro_average = f1_score(y_true=y_true, y_pred=y_pred, average='macro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1_micro': f1_micro_average,
               'f1_macro': f1_macro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [36]:
# Set the loss function
loss_fn = 'F1' # !!!

trainers_dict = {'BCE': Trainer, 'F1': F1Trainer, 'FL': FLTrainer, 'ASL': ASLTrainer, 'HL': HLTrainer, 'F1weight': F1weightTrainer, 'F1learn': F1learnTrainer} 

In [37]:
if loss_fn == 'F1learn':
#     model.register_parameter(name='weights', param=torch.nn.Parameter(torch.ones(52)))
    model.register_parameter(name='F1thr', param=torch.nn.Parameter(torch.zeros(52)))
    
trainer_class = trainers_dict[loss_fn]

In [ ]:
trainer = trainer_class(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics, 
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3000000)]
)

In [39]:
%%time
trainer.train(resume_from_checkpoint=True)
# trainer.train()

Loading model from scibert-cased/checkpoint-49000).
***** Running training *****
  Num examples = 303525
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 94860
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 5
  Continuing training from global step 49000
  Will skip the first 5 epochs then the first 1570 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command, but you will resume the training on data already seen by your model.


  0%|          | 0/1570 [00:00<?, ?it/s]

Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [41]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 50630
  Batch size = 32


KeyboardInterrupt: 

In [42]:
# Predict:
predictions = trainer.predict(encoded_dataset['test'])
print(predictions.predictions.shape, predictions.label_ids.shape)

***** Running Prediction *****
  Num examples = 151882
  Batch size = 32


KeyboardInterrupt: 

# Inference

In [ ]:
from tqdm.notebook import tqdm

with torch.no_grad():
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    model.to(device)
    test_dataloader = DataLoader(encoded_dataset["test"], shuffle=False, batch_size=8, collate_fn=data_collator)
    outputs = torch.Tensor()
    labels = torch.Tensor()
    for bi, batch in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        out = model(**batch)
        if len(outputs) == 0:
            outputs = out.logits
            labels = batch['labels']
        else:
            outputs = torch.cat((outputs, out.logits), 0)
            labels = torch.cat((labels, batch['labels']), 0)
print('Done')

  0%|          | 0/18986 [00:00<?, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [120]:
out = pd.DataFrame(outputs.cpu().numpy())
out.to_csv('out.csv')

In [121]:

threshold = 0.5
probs = sigmoid(outputs.to('cpu'))
probs
# next, use threshold to turn them into integer predictions
y_pred = np.zeros(probs.shape)
y_pred[np.where(probs >= threshold)] = 1

# finally, compute metrics
y_true = labels.cpu().numpy()
f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
f1_macro_average = f1_score(y_true=y_true, y_pred=y_pred, average='macro')
roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
accuracy = accuracy_score(y_true, y_pred)
# return as dictionary
metrics = {'f1_micro': f1_micro_average,
           'f1_macro': f1_macro_average,
           'roc_auc': roc_auc,
           'accuracy': accuracy}
metrics

{'f1_micro': 0.7926543505560374,
 'f1_macro': 0.7152693120064969,
 'roc_auc': 0.9043663335012946,
 'accuracy': 0.10022912524196416}

In [135]:
def get_metrics(preds, labels, thr=0.5):
    '''
    Create some metrics: precison, recall, F1...
    
    A macro-average will compute the metric independently for each class and then take the average hence 
    treating all classes equally, whereas a micro-average will aggregate the contributions of all classes
    to compute the average metric.
    '''
    
    # Convert the lists to dataframes
    lab_df = pd.DataFrame(labels.cpu().numpy())
    y_pred = np.zeros(preds.shape)
    y_pred[np.where(preds >= threshold)] = 1
    pred_df = pd.DataFrame(y_pred)
    
    # Calculate tp/fp/fn/tn per class:
    tp = (pred_df + lab_df).eq(2).sum()
    fp = (pred_df - lab_df).eq(1).sum()
    fn = (pred_df - lab_df).eq(-1).sum()
    tn = (pred_df + lab_df).eq(0).sum()
    
    # Calculate precision and recall:
    prec = [tp[i] / (tp[i] + fp[i]) if tp[i] + fp[i] != 0 else 0.0 for i in range(len(tp))]
    rec = [tp[i] / (tp[i] + fn[i]) if tp[i] + fn[i] != 0 else 0.0 for i in range(len(tp))]
    
    # Calculate F1 score:
    f1_score = [2 * prec[i] * rec[i] / (prec[i] + rec[i]) if tp[i] > 0 else 0.0 for i in range(len(tp))]
    
    # Weighted F1 score:
    weight = lab_df.sum() / sum(lab_df.sum())
    f1_wght = [weight[i] * 2 * prec[i] * rec[i] / (prec[i] + rec[i]) if tp[i] > 0 else 0.0 for i in range(len(tp))]
    
    # Macro average (average over classes):
    prec_avg = sum(prec) / len(prec)
    rec_avg = sum(rec) / len(rec)
    f1_avg = sum(f1_score) / len(f1_score)
    f1wgt_avg = sum(f1_wght)
    
    # Micro scores (treat all samples together):
    tp_mic = sum(tp)
    tn_mic = sum(tn)
    fp_mic = sum(fp)
    fn_mic = sum(fn)
    prec_mic = tp_mic / (tp_mic+fp_mic)
    rec_mic = tp_mic / (tp_mic+fn_mic)
    f1_mic = (2*prec_mic*rec_mic) / (prec_mic+rec_mic)
    
    return {
        'Precision': prec, 'Recall': rec, 'F1 score': f1_score,
        'weights': weight, 'Weighted F1 score': f1_wght,
        'Macro precision': prec_avg.round(2), 'Macro recall': rec_avg.round(2), 'Macro F1 score': f1_avg.round(2),
        'Weighted F1 score': f1wgt_avg.round(2),
        'CM TP': tp, 'CM FP': fp,'CM FN': fn, 'CM TN': tn,
        'Micro Precision': round(prec_mic, 2), 'Micro Recall': round(rec_mic, 2), 'Micro F1 score': round(f1_mic, 2),
    }



In [137]:
all_metrics = get_metrics(probs, labels)

for metr, val in all_metrics.items():
    if 'Micro' in metr or 'Macro' in metr:
        print(metr, val)

Macro precision 0.69
Macro recall 0.77
Macro F1 score 0.72
Micro Precision 0.74
Micro Recall 0.85
Micro F1 score 0.79
